In [1]:
# Install the vncorenlp python wrapper
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.6 MB 8.2 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=bfc72cf515ed824172b6af68793377b5cf32e1e78c711586685f6ed58015b92e
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
--2021-12-22 08:21:43--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   140MB/s    in 0.2s    

2021-12-22 08:21:44 (140 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2021-12-22 08:21:44-- 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/NLP/project_nlp/
%ls

/content/drive/MyDrive/NLP/project_nlp
app/  data/  model/  notebook/  paper/  PhoBERT_base_transformers/  src/


In [4]:
import pandas as pd 
import numpy as np
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [5]:
# Load rdrsegmenter from VnCoreNLP
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

# Input 
text = "Mặc cảm giác như là đồ chợ"

# To perform word (and sentence) segmentation
sentences = rdrsegmenter.tokenize(text)
print(sentences)
for sentence in sentences:
	print(" ".join(sentence))

[['Mặc_cảm', 'giác', 'như', 'là', 'đồ', 'chợ']]
Mặc_cảm giác như là đồ chợ


In [37]:
df = pd.read_excel('./data/clean/final6.xlsx')
df.drop(columns='Unnamed: 0', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3791 entries, 0 to 3790
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       3791 non-null   int64 
 1   text_prep2  3791 non-null   object
dtypes: int64(1), object(1)
memory usage: 59.4+ KB


In [38]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3673 entries, 0 to 3672
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       3673 non-null   int64 
 1   label       3673 non-null   int64 
 2   text_prep2  3673 non-null   object
dtypes: int64(2), object(1)
memory usage: 86.2+ KB


In [30]:
df.text_prep2[0]

'combo 3 cái giao có 1 cái , thành_ra đặt 6 cái nhận được 4 , hàng thì vải xấu giống vải áo_mưa , con trai chê vứt đi rồi'

In [ ]:
rdrsegmenter.tokenize(df.text_prep2[0])

In [32]:
dataset_size = len(df)
a = [len(df.text_prep2[i].split()) * [i] for i in range(dataset_size)]
out = []
for sublist in a:
    out.extend(sublist)
a = [df.text_prep2[i].split() for i in range(dataset_size)]

tokens = []
for sublist in a:
    tokens.extend(sublist)

initial_tags = ['O'] * len(out)


In [39]:
df_flat = pd.DataFrame()
df_flat['sentence'] = out
df_flat['tokens'] = tokens
df_flat['tag'] = initial_tags
df_flat['label'] = [0] * len(out)
df_flat

,sentence,tokens,tag,label
0,0,combo,O,0
1,0,3,O,0
2,0,cái,O,0
3,0,giao,O,0
4,0,có,O,0
...,...,...,...,...
87456,3672,.,O,0
87457,3672,tiki,O,0
87458,3672,giao,O,0
87459,3672,hàng,O,0


In [ ]:
def process_term(path):
    new_path = path.replace('.txt', '_tokens.txt')
    with open(path, 'r') as f:
        price_terms = f.read().splitlines()

    with open(new_path, 'w') as f:
        for term in price_terms:
            token = rdrsegmenter.tokenize(term)
            f.write(' '.join(token[0])+'\n')

def get_term(path, check=False):
    new_path = path.replace('.txt', '_tokens.txt') 
    with open(new_path, 'r') as f:
        price_terms = f.read().splitlines()
    outputs = sorted(price_terms, key=lambda x: len(x.split()))
    if check:
        print(outputs)
    return outputs

In [ ]:
# process_term('./data/seq_tag/design_term.txt')
process_term('./data/seq_tag/color_term.txt')
process_term('./data/seq_tag/price_term.txt')

# design_terms = get_term('./data/seq_tag/design_term.txt', check=True)
color_terms = get_term('./data/seq_tag/color_term.txt', check=True)
price_terms = get_term('./data/seq_tag/price_term.txt', check=True)

# design_terms.reverse()
color_terms.reverse()
price_terms.reverse()

# design_df = pd.DataFrame(design_terms, columns=['terms'])
# design_df['tags'] = ['O'] * len(design_df)
# design_df.to_csv('./data/seq_tag/design_term_tags.csv')

color_df = pd.DataFrame(color_terms, columns=['terms'])
color_df['tags'] = ['O'] * len(color_df)
color_df.to_csv('./data/seq_tag/color_term_tags.csv')


price_df = pd.DataFrame(price_terms, columns=['terms'])
price_df['tags'] = ['O'] * len(price_df)
price_df.to_csv('./data/seq_tag/price_term_tags.csv')

['hoạ_tiết', 'màu_sắc', 'be', 'bạc', 'cam', 'caro', 'chi_tiết', 'hoa_văn', 'hồng', 'hoạ_tiết', 'hồng', 'in_ấn', 'kem', 'ren', 'màu', 'màu_da', 'màu_sắc', 'mầu', 'mẫu_mã', 'nhuộm', 'nude', 'pastel', 'phối_ren', 'sặc_sỡ', 'sậm', 'tím', 'trắng', 'trắng_ngà', 'tím', 'tươi_sáng', 'xanh', 'xám', 'đỏ', 'be nhạt', 'be đậm', 'chấm bi', 'hồng nhạt', 'hồng đậm', 'in chữ', 'in hình', 'màu hồng', 'màu trắng', 'màu vàng', 'màu be', 'màu bạc', 'màu hồng', 'màu kem', 'màu nude', 'màu nâu', 'màu pastel', 'màu quần', 'màu rêu', 'màu trắng', 'màu tối', 'màu vàng', 'màu xanh', 'màu xám', 'màu áo', 'màu đen', 'màu đậm', 'màu đỏ', 'mầu da', 'mầu kem', 'mầu nâu', 'mầu quần', 'mầu trắng', 'mầu vàng', 'mầu áo', 'nhiều màu', 'nâu đất', 'nét in', 'ra màu', 'tone màu', 'tông màu', 'tông mầu', 'xanh biển', 'xanh coban', 'xanh lá', 'xanh nhạt', 'xanh đậm', 'xám trắng', 'có chữ', 'đỏ gạch', 'đỏ sậm', 'đỏ tươi', 'đỏ đô', 'màu hồng nhạt', 'màu hồng đậm', 'màu kem nhạt', 'màu kem đậm', 'màu nâu nhạt', 'màu nâu 

In [ ]:
tokens_list = list(df_flat['tokens'].values)
len(tokens_list)

90140

In [ ]:
def subfinder(mylist, pattern):
    matches = []
    indexes = []
    for i in range(len(mylist)):
        if mylist[i] == pattern[0] and mylist[i:i+len(pattern)] == pattern:
            matches.append(pattern)
            indexes.append(i)
    return matches, indexes

In [ ]:
matches, indexes = subfinder(tokens_list, ['3xl'])
indexes

[2347,
 2362,
 3916,
 6142,
 8265,
 11027,
 11660,
 11757,
 11759,
 13786,
 18319,
 19356,
 19628,
 25009,
 33193,
 33608,
 42490,
 46501,
 52875,
 52889,
 52945,
 57850,
 58976,
 59813,
 61317,
 67328,
 72901,
 76303]

In [ ]:
label_dict = {
    'O': 0,
    'B-DES': 1,
    'I-DES': 2,
    'B-PRI': 3,
    'I-PRI': 4
}

# label_dict = {
#     'O': 0,
#     'B-COL': 1,
#     'I-COL': 2,
#     'B-PRI': 3,
#     'I-PRI': 4
# }

In [ ]:
design_tags = pd.read_excel('./data/seq_tag/design_term_tags.xlsx')
# color_tags = pd.read_excel('./data/seq_tag/color_term_tags.xlsx')
price_tags = pd.read_excel('./data/seq_tag/price_term_tags.xlsx')
design_tags.drop(columns=['Unnamed: 0'], inplace=True)
# color_tags.drop(columns=['Unnamed: 0'], inplace=True)
price_tags.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
def mapping_tags(terms_tags):
    for i in range(len(terms_tags)-1, 0, -1):
    # for i in range(len(terms_tags)):
        terms = terms_tags.iloc[i].terms.split()
        # print('Process tearm:', terms)
        matches, indexes = subfinder(tokens_list, terms)
        if not indexes:
            # print('No matches')
            pass
        else:
            # print('Matches')
            print('Match terms', terms)
            print('Number of match tearms: ', len(indexes))
            for j in indexes:
                print('Index: ', j)
                for k in range(len(terms)):
                    df_flat['tag'].iloc[j+k] = terms_tags.iloc[i].tags.split(',')[k].strip()
                    print('Map token {} tag from {} to {}'.format(df_flat['tag'].iloc[j+k], terms[k], terms_tags.iloc[i].tags.split(',')[k].strip()))
        # break
mapping_tags(design_tags)
# mapping_tags(color_tags)
mapping_tags(price_tags)

Match terms ['2xl']
Number of match tearms:  22
Index:  5490
Map token B-DES tag from 2xl to B-DES
Index:  11026
Map token B-DES tag from 2xl to B-DES
Index:  11755
Map token B-DES tag from 2xl to B-DES
Index:  13402
Map token B-DES tag from 2xl to B-DES
Index:  13998
Map token B-DES tag from 2xl to B-DES
Index:  23756
Map token B-DES tag from 2xl to B-DES
Index:  28959
Map token B-DES tag from 2xl to B-DES
Index:  31224
Map token B-DES tag from 2xl to B-DES
Index:  34886
Map token B-DES tag from 2xl to B-DES
Index:  35881
Map token B-DES tag from 2xl to B-DES
Index:  37877
Map token B-DES tag from 2xl to B-DES
Index:  38937
Map token B-DES tag from 2xl to B-DES
Index:  39648
Map token B-DES tag from 2xl to B-DES
Index:  40489
Map token B-DES tag from 2xl to B-DES
Index:  42843
Map token B-DES tag from 2xl to B-DES
Index:  43012
Map token B-DES tag from 2xl to B-DES
Index:  52577
Map token B-DES tag from 2xl to B-DES
Index:  52856
Map token B-DES tag from 2xl to B-DES
Index:  58979
Map

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Streaming output truncated to the last 5000 lines.
Index:  5001
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  5691
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  6491
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  6724
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  9336
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  9757
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  10897
Map token B-DES tag from không to B-DES
Map token I-DES tag from giống to I-DES
Map token I-DES tag from hình to I-DES
Index:  12564
Map token 

In [43]:
df_flat

,sentence,tokens,tag,label
0,0,combo,O,-1
1,0,3,O,-1
2,0,cái,O,-1
3,0,giao,O,-1
4,0,có,O,-1
...,...,...,...,...
87456,3672,.,O,1
87457,3672,tiki,O,1
87458,3672,giao,O,1
87459,3672,hàng,O,1


In [45]:
df

,index,label,text_prep2
0,0,-1,"combo 3 cái giao có 1 cái , thành_ra đặt 6 cái..."
1,1,-1,mình mua áo có cổ màu trắng lại ship tới cho m...
2,2,-1,giao sai hàng . tôi muốn trả hàng . đặt be đậm...
3,3,-1,sản_xuất việt_nam nhưng thấy in chữ trung_quốc...
4,4,-1,mình đặt áo sơ_mi trắng dài tay mà shop giao c...
...,...,...,...
3668,3786,1,chất_lượng sản_phẩm giống mô tả . giao hàng nh...
3669,3787,1,"hài_lòng vô_cùng , giao nhanh , nhân_viên giao..."
3670,3788,1,sản_phẩm ổn . giá phải_chăng . thật_sự là nhận...
3671,3789,1,hàng đẹp chuẩn chất_lượng . nếu áo có đai ngan...


In [41]:
df_flat['label'] = [df['label'][int(i)] for i in df_flat.sentence.values]
df_flat

,sentence,tokens,tag,label
0,0,combo,O,-1
1,0,3,O,-1
2,0,cái,O,-1
3,0,giao,O,-1
4,0,có,O,-1
...,...,...,...,...
87456,3672,.,O,1
87457,3672,tiki,O,1
87458,3672,giao,O,1
87459,3672,hàng,O,1


In [42]:
df_flat.sentence.drop_duplicates().shape

(3673,)

In [44]:
df_flat.to_csv('./data/seq_tag/tokens_labeled_sentiment.csv')

### Xử lý các từ trong while list, các số tiền sử dụng regex

In [ ]:
# process_term('./data/seq_tag/color_whitelist.txt')
process_term('./data/seq_tag/price_whitelist.txt')
process_term('./data/seq_tag/design_whitelist.txt')

price_whitelist = get_term('./data/seq_tag/price_whitelist.txt', check=True)
design_whitelist = get_term('./data/seq_tag/design_whitelist.txt', check=True)


['cảm_giác', 'giá_mà', 'lãng_phí', 'đánh_giá', 'bỏ phí', 'nhét tiền', 'rẻ lau chân']
['bé_con', 'bó_tay', 'bạn_bè', 'chất_lượng', 'chậm_trễ', 'con_nhỏ', 'cạn_lời', 'dài_dòng', 'dây_thần_kinh', 'hoá_chất', 'lâu_dài', 'nhỏ_con', 'thân_thiết', 'thân_thiện', 'thân_ái', 'thô_lỗ', 'tính_chất', 'từ_khoá', 'xin_lỗi', 'bé lớn', 'bảng size', 'chiều cao', 'cho bé', 'chỉ có', 'chỉ giao', 'chỉ là', 'còn trễ', 'dáng người', 'em bé', 'giao trễ', 'giá mềm', 'gói túi', 'khoá bài', 'lỗi của', 'lỗi hệ_thống', 'lớp 4', 'người cao', 'người lớn', 'người thân', 'ngực bé', 'ngực nhỏ', 'sen đỏ', 'sáng giờ', 'sáng hôm_qua', 'sáng nay', 'thương_hiệu lớn', 'tiền dư', 'trễ hẹn', 'tôn dáng', 'đóng_gói túi', 'đóng_gói đẹp', 'đóng túi', 'giao hàng trễ']


In [ ]:
price_whitelist = [word.replace('_', ' ') for word in price_whitelist]
price_whitelist

In [ ]:
_, index = subfinder(tokens_list, ['bỏ'])
df_flat.iloc[index]
index = [i for i in index]
df_flat.iloc[index]

In [ ]:
index = [i-1 for i in index]

In [ ]:
df = pd.read_csv('./data/seq_tag/tokens_labeled_no_whitelist_final6.csv')
df.head()

,Unnamed: 0,sentence,tokens,tag
0,0,0,combo,O
1,1,0,3,O
2,2,0,cái,O
3,3,0,giao,O
4,4,0,có,O


In [ ]:
df1 = df['tokens'].values
all_sentences = ' '.join(df1)

In [ ]:
pat = '(\d+k)|(\d+ k)|(\d+ ngàn đồng)|(\d+ nghìn đồng)|(\d+ ngàn)|(\d+ nghìn)|(\d+ \d+ nghìn)|(\d+ \d+ ngàn)|(\d+ \d+ k)|(\d+ \d+k)|(\d+ \d+nghìn)|(\d+ \d+ngàn)|(\d+d)|(\d+ d)|(\d+đ)|(\d+ đồng)|(\d+ đ)'

In [ ]:
import re
re.findall(r'lớp 4', all_sentences)

['lớp 4']